This is a conversion into notebook format of the script `lisfloodreservoirs.calibrate.py`.

Description:

```Text
Run the calibration script with a specified configuration file.
It calibrates the reservoir model parameters of the defined routine using the
SCE-UA (Shuffle Complex Evolution-University of Arizona) algorithm for each of
the selected reservoirs.
The optimal parameters are simulated and plotted, if possible comparing against
a simulation with default parameters
```

In [1]:
# Path to the configuration file
CONFIG_FILE = './camaflood_2var.yml'
# Overwrite existing simulation file
OVERWRITE = False

In [2]:
# %load ../../../src/lisfloodreservoirs/calibrate.py
#!/usr/bin/env python3

import argparse
import logging
import yaml
import pickle
from pathlib import Path

import pandas as pd
from tqdm import tqdm
import spotpy
from datetime import datetime

from lisfloodreservoirs import Config, read_attributes, read_timeseries
from lisfloodreservoirs.models import get_model
from lisfloodreservoirs.calibration import get_calibrator, read_results
from lisfloodreservoirs.utils.metrics import KGEmod, compute_performance
from lisfloodreservoirs.utils.timeseries import create_demand
from lisfloodreservoirs.utils.logging import setup_logger

In [3]:
# === Configuration ===

# read configuration file
cfg = Config(CONFIG_FILE) #Config(args.config_file)

# set up logger
logger = setup_logger(
    name=__name__,
    log_level=logging.INFO,
    log_file=None #f'{datetime.now():%Y%m%d%H%M}_calibrate_{cfg.MODEL}.log'
)

logger.info(f'Calibration results will be saved in: {cfg.PATH_CALIB}')

2025-05-28 09:14:27 | INFO | __main__ | Calibration results will be saved in: Z:\nahaUsers\casadje\datasets\reservoirs\ResOpsUS\v2.1\results\camaflood\calibration\bivariate
INFO: 4197965295.<module>(): Calibration results will be saved in: Z:\nahaUsers\casadje\datasets\reservoirs\ResOpsUS\v2.1\results\camaflood\calibration\bivariate


In [4]:
# === Load reservoir list ===
try:
    reservoirs = pd.read_csv(cfg.RESERVOIRS_FILE, header=None).squeeze().tolist()
except IOError:
    logger.exception(f'Failed to open {cfg.RESERVOIRS_FILE}')
    raise

In [5]:
# === Load attributes ===
try:
    attributes = read_attributes(cfg.PATH_DATA / 'attributes', reservoirs)
    logger.info(f'{attributes.shape[0]} reservoirs in the attribute tables')
except IOError:
    logger.exception('Failed to read attribute tables from {0}'.format(cfg.PATH_DATA / 'attributes'))
    raise

2025-05-28 09:15:00 | INFO | __main__ | 164 reservoirs in the attribute tables
INFO: 3498897491.<module>(): 164 reservoirs in the attribute tables


In [6]:
# === Load time periods ===
try:
    with open(cfg.PERIODS_FILE, 'rb') as file:
        periods = pickle.load(file)
except IOError:
    logger.exception(f'Failed to open {cfg.PERIODS_FILE}')
    raise

In [7]:
# === read time series ===
try:
    inputs = [var for var in [cfg.INFLOW, cfg.PRECIPITATION, cfg.EVAPORATION, cfg.DEMAND] if var]
    outputs = ['storage', 'outflow']
    timeseries = read_timeseries(
        path=cfg.PATH_DATA / 'time_series' / 'csv',
        reservoirs=attributes.index,
        periods=periods,
        variables=inputs + outputs
    )
    logger.info(f'{len(timeseries)} reservoirs with timeseries')
except IOError:
    logger.exception('Failed to read time series from {0}'.format(cfg.PATH_DATA / 'time_series' / 'csv'))
    raise

  0%|          | 0/164 [00:00<?, ?it/s]

2025-05-28 09:15:44 | INFO | __main__ | 164 reservoirs with timeseries
INFO: 2671642390.<module>(): 164 reservoirs with timeseries


In [ ]:
cfg

In [8]:
# === Calibration ===
for grand_id, ts in tqdm(timeseries.items(), desc='simulating reservoir'):
        
    dbname = f'{cfg.PATH_CALIB}/{grand_id}_samples'
    if Path(f'{dbname}.csv').is_file() and not OVERWRITE: #args.overwrite:
        logger.info(f'Calibration already exists for reservoir {grand_id}, skipping (use --overwrite to force)')
        continue
    
    logger.info(f'Calibrating reservoir {grand_id}')
    
    # define input time series
    inflow = ts[cfg.INFLOW]
    precipitation = ts[cfg.PRECIPITATION] if cfg.PRECIPITATION in ts.columns else None
    evaporation = ts[cfg.EVAPORATION] if cfg.EVAPORATION in ts.columns else None
    demand = ts[cfg.DEMAND] if cfg.DEMAND in ts.columns else None
    if cfg.MODEL == 'mhm':
        bias = ts.outflow.mean() / inflow.mean()
        demand = create_demand(
            ts.outflow,
            water_stress=min(1, bias),
            window=28
        )
        
    # storage attributes (m3)
    Vtot = ts.storage.max()
    Vmin = max(0, min(0.1 * Vtot, ts.storage.min()))
    # flow attributes (m3/s)
    Qmin = max(0, ts.outflow.min())
    # catchment area (m2)
    catchment = int(attributes.loc[grand_id, 'CATCH_SKM'] * 1e6) if cfg.MODEL == 'camaflood' else None
    # reservoir area (m2)
    Atot = int(attributes.loc[grand_id, 'AREA_SKM'] * 1e6)

    # calibrate
    try:
        # configure calibration kwargs
        cal_cfg = {}
        if cfg.MODEL == 'camaflood':
            cal_cfg.update({'catchment': catchment})
        # elif cfg.MODEL == 'mhm':
        #     cal_cfg.update({'demand': demand})
        # initialize the calibration setup
        calibrator = get_calibrator(
            cfg.MODEL,
            inflow=inflow,
            storage=ts.storage,
            outflow=ts.outflow,
            precipitation=precipitation,
            evaporation=evaporation,
            demand=demand,
            Vmin=Vmin,
            Vtot=Vtot,
            Qmin=Qmin,
            Atot=Atot,
            target=cfg.TARGET,
            obj_func=KGEmod,
            spinup=cfg.SPINUP,
            **cal_cfg
        )
        # define the sampling method
        sceua = spotpy.algorithms.sceua(
            calibrator, 
            dbname=dbname, 
            dbformat='csv', 
            # parallel='mpi',
            save_sim=False,
            # seed=42
        )
        # launch calibration
        sceua.sample(
            cfg.MAX_ITER, 
            ngs=cfg.COMPLEXES, 
            kstop=3, 
            pcento=0.01, 
            peps=0.1
        )
        logger.info(f'Calibration of reservoir {grand_id} successfully finished')
    except RuntimeError:
        logger.exception(f'Reservoir {grand_id} could not be calibrated')
        continue
        
    # simulate optimized reservoir
    try:
        
        # read calibration results
        results, parameters = read_results(f'{dbname}.csv')
        
        # convert parameter into reservoir attributes
        calibrated_attrs = calibrator.pars2attrs(list(parameters.values()))

        # declare the reservoir with optimal parameters
        res = get_model(cfg.MODEL, **calibrated_attrs)

        # export calibrated parameters
        with open(cfg.PATH_CALIB / f'{grand_id}_optimal_parameters.yml', 'w') as file:
            yaml.dump(res.get_params(), file)

        # simulate the reservoir
        # sim_cfg = {} if cfg.MODEL == 'camaflood' else cal_cfg
        sim_cal = res.simulate(
            inflow=inflow,
            Vo=ts.storage.iloc[0],
            precipitation=precipitation,
            evaporation=evaporation,
            demand=demand,
            # **sim_cfg
        )
        sim_cal.to_csv(cfg.PATH_CALIB / f'{grand_id}_simulation.csv', float_format='%.3f')
        
        logger.info(f'Simulation of the calibrated reservoir {grand_id} successfully finished')
        
    except RuntimeError:
        logger.exception(f'Calibrated reservoir {grand_id} could not be simulated')
        continue
        
    # === Analyse results ===
    
    # performance
    try:
        performance_cal = compute_performance(ts.iloc[cfg.SPINUP:], sim_cal.iloc[cfg.SPINUP:])
        performance_cal.to_csv(cfg.PATH_CALIB / f'{grand_id}_performance.csv', float_format='%.3f')
        logger.info(f'Performance of reservoir {grand_id} has been computed')
    except IOError:
        logger.exception(f'The performance of reservoir {grand_id} could not be exported')
        
    # scatter plot calibration vs observation
    try:
        res.scatter(
            sim_cal,
            ts,
            spinup=cfg.SPINUP,
            norm=False,
            title=f'grand_id: {grand_id}',
            save=cfg.PATH_CALIB / f'{grand_id}_scatter.jpg',
        )
        logger.info(f'Scatter plot of simulation from reservoir {grand_id}')
    except IOError:
        logger.exception(f'The scatter plot of reservoir {grand_id} could not be generated')
        
    # line plot calibration (vs default simulation) vs observation
    try:
        file_default = cfg.PATH_DEF / f'{grand_id}_simulation.csv'
        if file_default.is_file():
            sim_def = pd.read_csv(cfg.PATH_DEF / f'{grand_id}_simulation.csv', parse_dates=True, index_col=0)
            sim = {
                'default': sim_def,
                'calibrated': sim_cal
            }
        else:
            sim = {'calibrated': sim_cal}
        res.lineplot(
            sim,
            ts,
            spinup=cfg.SPINUP,
            figsize=(12, 6),
            save=cfg.PATH_CALIB / f'{grand_id}_line.jpg',
        )
        logger.info(f'Line plot of simulation from reservoir {grand_id}')
    except IOError:
        logger.exception(f'The line plot of reservoir {grand_id} could not be generated')
        
    del res, calibrator, sim_cal, calibrated_attrs, performance_cal#, sim_cfg
    try:
        del sceua
    except:
        pass

simulating reservoir:   0%|                                                                      | 0/5 [00:00<?, ?it/s]2025-05-08 13:45:20 | INFO | __main__ | Calibrating reservoir 41
INFO: 3828140465.<module>(): Calibrating reservoir 41


Initializing the  Shuffled Complex Evolution (SCE-UA) algorithm  with  1000  repetitions
The objective function will be minimized
Starting burn-in sampling...


timesteps:   0%|          | 0/7544 [00:00<?, ?it/s]

Initialize database...
['csv', 'hdf5', 'ram', 'sql', 'custom', 'noData']
* Database file 'Z:\nahaUsers\casadje\datasets\reservoirs\ResOpsUS\v2.0\test\no_meteo\camaflood\calibration\bivariate/41_samples.csv' created.


timesteps:   0%|          | 0/7544 [00:00<?, ?it/s]

...

timesteps:   0%|          | 0/10442 [00:00<?, ?it/s]

2025-05-08 14:07:37 | INFO | __main__ | Calibration of reservoir 1753 successfully finished
INFO: 3828140465.<module>(): Calibration of reservoir 1753 successfully finished


1093 of 1000, minimal objective function=0.843272, time remaining: 23:59:38
Skipping saving
Skipping saving
Skipping saving
Skipping saving
Skipping saving
Skipping saving
Skipping saving
Skipping saving
Skipping saving
Skipping saving
Skipping saving
*** OPTIMIZATION SEARCH TERMINATED BECAUSE THE LIMIT
ON THE MAXIMUM NUMBER OF TRIALS 
1000
HAS BEEN EXCEEDED.
SEARCH WAS STOPPED AT TRIAL NUMBER: 1103
NUMBER OF DISCARDED TRIALS: 41
NORMALIZED GEOMETRIC RANGE = 0.605171
THE BEST POINT HAS IMPROVED IN LAST 3 LOOPS BY 0.775683 PERCENT

*** Final SPOTPY summary ***
Total Duration: 257.17 seconds
Total Repetitions: 1103
Minimal objective value: 0.843272
Corresponding parameter setting:
alpha: 0.548799
beta: 0.0148546
gamma: 0.69314
delta: 0.390825
epsilon: 0.873768
******************************



timesteps:   0%|          | 0/10442 [00:00<?, ?it/s]

2025-05-08 14:07:37 | INFO | __main__ | Simulation of the calibrated reservoir 1753 successfully finished
INFO: 3828140465.<module>(): Simulation of the calibrated reservoir 1753 successfully finished
2025-05-08 14:07:37 | INFO | __main__ | Performance of reservoir 1753 has been computed
INFO: 3828140465.<module>(): Performance of reservoir 1753 has been computed
2025-05-08 14:07:38 | INFO | __main__ | Scatter plot of simulation from reservoir 1753
INFO: 3828140465.<module>(): Scatter plot of simulation from reservoir 1753
2025-05-08 14:07:39 | INFO | __main__ | Line plot of simulation from reservoir 1753
INFO: 3828140465.<module>(): Line plot of simulation from reservoir 1753
simulating reservoir: 100%|█████████████████████████████████████████████████████████████| 5/5 [22:18<00:00, 267.74s/it]
